<a href="https://colab.research.google.com/github/athiniotim/Final_Essay_Forest_Fires/blob/main/forest_fires_athensvoice_articles_scraper_Michaela_Athinioti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    # print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):

  article_block = soup.find("div", {"class": "articleSingle articleSingle--default"})
  # article url
  article_url = article_url
  # title
  try:
    title = article_block.find("h1", {"class": "articleSingle__title"}).text.strip()
  except:
    # try:
    #   title = soup.find("div", {"class": "container is-fullhd p-3"}).h1.text.strip() # προσθήκη για να πιάνει και τον τίτλο όταν βρίσκεται σε αυτό το div
    # except:
    title=""
  # datetime
  try:
    datetime_str = article_block.find("time")["datetime"]
  except:
    try:
      datetime_str = soup.find("li", {"class": "actionsList__item actionsList__item--date"}).find("time")["datetime"]
    except:
      datetime_str = ""
      date_str = ""
      time_str = ""
  if datetime_str != "":
    date_str = datetime_str.split("T")[0]
    time_str = datetime_str.split("T")[1].split("Z")[0]
  # section
  try:
    section = article_block.find("a", class_=lambda c: c and "articleSingle__category" in c).text.strip()
  except:
    section = ""
  # author
  try:
    author = article_block.find("a", class_=lambda c: c and "actionsList__link" in c).find("span").text.strip()
  except:
    # try:
    #   author = soup.find("div", {"class": "container is-fullhd p-3"}).find("a", {"rel": "author"}).text.strip() # προσθήκη
    # except:
    author = ""
  #subtitle
  try:
    subtitle = article_block.find("div", {"class": "articleSingle__intro"}).text.strip()
  except:
    subtitle = ""
  # excerpt
  try:
    excerpt = article_block.find("div", {"class": "articleSingle__summaryContent"}).find("p").text.strip()
  except:
    # try:
    #   excerpt = soup.find("div", {"class": "container is-fullhd p-3"}).find("div", {"class": "newsletter-description nx-excerpt my-5"}).text.strip() # προσθήκη
    # except:
    excerpt=""
  # text body
  try:
    text_body_p_tagsL = article_block.find("div", class_="articleSingle__description").find_all("p")
    text_body_textsL = []
    for p_tag in text_body_p_tagsL:
      p_tag_text = p_tag.text.strip()
      text_body_textsL.append(p_tag_text)
    text_body = " ".join(text_body_textsL)
  except:
    text_body=""
  # feature image
  try:
    feat_img_src = article_block.find("div", class_="articleSingle__image").find('img')["src"]
    feat_img_link = "https://www.athensvoice.gr"+feat_img_src
  except:
    feat_img_link=""


  articleD = {"article_url": article_url, "title": title, "date": date_str, "time": time_str, "section": section, "author": author,
              "excerpt": excerpt, "subtitle": subtitle, "text_body": text_body, "feat_img_link": feat_img_link}

  return articleD

In [ ]:
def scrape_athensvoice_articles_pages(teasers_df):
  all_articles_dataL = []
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):

    article_url = row['article_url']
    soup = get_webpage_soup(article_url)
    if soup is not None:
      articleD = get_article_elements_from_soup(soup, article_url)
      all_articles_dataL.append(articleD)
    else:
      # Optional: Print or log the URL that failed to scrape
      print(f"Warning: Could not scrape {article_url}")
    # articleD = get_article_elements_from_soup(soup, article_url)
    # all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetched {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Δοκιμή του scraper**

In [ ]:
teasers_df = pd.read_csv("/content/drive/MyDrive/Final Essay ΕΔΔΕ.2 Αθηνιώτη Μιχαέλα/clean csv/forest_fires_athensvoice_teaser_csv.csv")
teasers_df.head()

,index,title,article_url,date,time,author,excerpt,section,date_dt
0,123,Μαγικές εικόνες από την «Πόλη του Πάγου» της Κ...,https://www.athensvoice.gr/epikairotita/diethn...,2024-12-29,10:40:50,Newsroom,Ουρανοξύστες και ναοί φτιαγμένοι από πάγο στο ...,Κοσμος,2024-12-29
1,124,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,https://www.athensvoice.gr/epikairotita/diethn...,2024-12-27,11:26:13,Newsroom,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,Κοσμος,2024-12-27
2,125,Η Ρωσία ξεκίνησε τη χρήση bitcoin για να αντιμ...,https://www.athensvoice.gr/epikairotita/diethn...,2024-12-25,15:02:24,Newsroom,Ο Ρώσος Πρόεδρος Βλαντιμίρ Πούτιν δήλωσε ότι κ...,Κοσμος,2024-12-25
3,126,Podcast 40+ | Ποιες είναι οι ιδανικές γιορτές ...,https://look.athensvoice.gr/look/opinions/8831...,2024-12-22,10:19:13,Μανίνα Ζουμπουλάκη - Ελένη Ψυχούλη,NaN,Opinion,2024-12-22
4,127,Titania: Μαγικές γιορτές με θέα τη στολισμένη ...,https://www.athensvoice.gr/life/geusi/themata/...,2024-12-20,16:17:06,A.V. Team,Titania: αξέχαστες γιορτές σε ένα από τα πιο ε...,Θεματα,2024-12-20


In [ ]:
teasers_df_test = teasers_df.head(10)

In [ ]:
articles_df = scrape_athensvoice_articles_pages(teasers_df_test)
articles_df

Processing rows: 100%|██████████| 10/10 [00:33<00:00,  3.40s/it]


Scraping completed. Fetched 10 articles.


,article_url,title,date,time,section,author,excerpt,subtitle,text_body,feat_img_link
0,https://www.athensvoice.gr/epikairotita/diethn...,Μαγικές εικόνες από την «Πόλη του Πάγου» της Κ...,2024-12-29,10:40:50,Κοσμος,Newsroom,Ουρανοξύστες και ναοί φτιαγμένοι από πάγο στο ...,Το πάρκο έχει έκταση ενός εκατομμυρίου τετραγω...,"Διάφορα κτίρια-ορόσημα του Χονγκ Κονγκ, του Μα...",https://www.athensvoice.gr/images/1074x600/jpg...
1,https://www.athensvoice.gr/epikairotita/diethn...,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,2024-12-27,11:26:13,Κοσμος,Newsroom,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,Φόβοι για την ασφάλεια των πτήσεων,Η Azerbaijan Airlines ανακοίνωσε ότι διακόπτει...,https://www.athensvoice.gr/images/1074x600/1/j...
2,https://www.athensvoice.gr/epikairotita/diethn...,Η Ρωσία ξεκίνησε τη χρήση bitcoin για να αντιμ...,2024-12-25,15:02:24,Κοσμος,Newsroom,Ο Ρώσος Πρόεδρος Βλαντιμίρ Πούτιν δήλωσε ότι κ...,Ο Βλαντιμίρ Πούτιν υποστηρίζει την εκτεταμένη ...,Εταιρείες στη Ρωσία ξεκίνησαν να χρησιμοποιούν...,https://www.athensvoice.gr/images/1074x600/1/j...
3,https://look.athensvoice.gr/look/opinions/8831...,,2024-12-21,09:30:00,,,,,,
4,https://www.athensvoice.gr/life/geusi/themata/...,,2024-12-20,16:17:06,,,Titania: αξέχαστες γιορτές σε ένα από τα πιο ε...,,Η ομορφότερη εποχή του χρόνου έφτασε και η Αθή...,
5,https://www.athensvoice.gr/epikairotita/politi...,Μιλένα Αποστολάκη προς Τάκη Θεοδωρικάκο: Μαζικ...,2024-12-19,10:11:59,Πολιτικη & Οικονομια,Newsroom,Μιλένα Αποστολάκη προς Τάκη Θεοδωρικάκο: Μαζικ...,Η βουλευτής Βόρειου Τομέα του ΠΑΣΟΚ σημειώνει ...,"Το μείζον θέμα των υπέρογκων, διαδοχικών και α...",https://www.athensvoice.gr/images/1074x600/1/j...
6,https://www.athensvoice.gr/life/perivallon/882...,Πόσο αυξήθηκαν οι πυρκαγιές το 2024 - Η ανασκό...,2024-12-17,18:12:54,Περιβαλλον,Newsroom,Πόσο αυξήθηκαν οι πυρκαγιές το 2024 - Η ανασκό...,Εθνικό Αστεροσκοπείο Αθηνών και WWF Ελλάς εγκα...,Η ολοκληρωμένη ανασκόπηση της αντιπυρικής περι...,https://www.athensvoice.gr/images/1074x600/jpg...
7,https://www.athensvoice.gr/epikairotita/politi...,Μητσοτάκης: Μηδενίζονται οι χρεώσεις για βασικ...,2024-12-16,07:52:00,Πολιτικη & Οικονομια,Newsroom,Οι ανακοινώσεις του Κυριάκου Μητσοτάκη για τις...,Όσα ανακοίνωσε ο πρωθυπουργός,Όλες οι χρεώσεις για βασικές τραπεζικές κινήσε...,https://www.athensvoice.gr/images/1074x600/1/j...
8,https://www.athensvoice.gr/epikairotita/diethn...,Ανασκόπηση 2024: Οι μεγαλύτερες φυσικές καταστ...,2024-12-10,05:51:44,Κοσμος,Γεωργία Ζερβογιάννη,Ανασκόπηση 2024: Οι μεγαλύτερες φυσικές καταστ...,Από τις φωτιές μέχρι τους σεισμούς και τις πλη...,Μέρες που είναι συνηθίζονται οι ανασκοπήσεις κ...,https://www.athensvoice.gr/images/1074x600/jpg...
9,https://www.athensvoice.gr/life/geusi/resto/87...,,2024-12-01,11:25:52,,,,,Αυτή την αυλή την αγαπούν πολύ οι Αθηναίοι. Γι...,


In [ ]:
fire_articles_df = scrape_athensvoice_articles_pages(teasers_df)
fire_articles_df

Processing rows:  97%|█████████▋| 623/642 [35:25<01:15,  3.96s/it]

Processing rows: 100%|██████████| 642/642 [36:32<00:00,  3.42s/it]


Scraping completed. Fetched 641 articles.


,article_url,title,date,time,section,author,excerpt,subtitle,text_body,feat_img_link
0,https://www.athensvoice.gr/epikairotita/diethn...,Μαγικές εικόνες από την «Πόλη του Πάγου» της Κ...,2024-12-29,10:40:50,Κοσμος,Newsroom,Ουρανοξύστες και ναοί φτιαγμένοι από πάγο στο ...,Το πάρκο έχει έκταση ενός εκατομμυρίου τετραγω...,"Διάφορα κτίρια-ορόσημα του Χονγκ Κονγκ, του Μα...",https://www.athensvoice.gr/images/1074x600/jpg...
1,https://www.athensvoice.gr/epikairotita/diethn...,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,2024-12-27,11:26:13,Κοσμος,Newsroom,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,Φόβοι για την ασφάλεια των πτήσεων,Η Azerbaijan Airlines ανακοίνωσε ότι διακόπτει...,https://www.athensvoice.gr/images/1074x600/1/j...
2,https://www.athensvoice.gr/epikairotita/diethn...,Η Ρωσία ξεκίνησε τη χρήση bitcoin για να αντιμ...,2024-12-25,15:02:24,Κοσμος,Newsroom,Ο Ρώσος Πρόεδρος Βλαντιμίρ Πούτιν δήλωσε ότι κ...,Ο Βλαντιμίρ Πούτιν υποστηρίζει την εκτεταμένη ...,Εταιρείες στη Ρωσία ξεκίνησαν να χρησιμοποιούν...,https://www.athensvoice.gr/images/1074x600/1/j...
3,https://look.athensvoice.gr/look/opinions/8831...,,2024-12-21,09:30:00,,,,,,
4,https://www.athensvoice.gr/life/geusi/themata/...,,2024-12-20,16:17:06,,,Titania: αξέχαστες γιορτές σε ένα από τα πιο ε...,,Η ομορφότερη εποχή του χρόνου έφτασε και η Αθή...,
...,...,...,...,...,...,...,...,...,...,...
636,https://www.athensvoice.gr/epikairotita/politi...,ΤΧΣ: Οι βασικές αρχές της στρατηγικής αποεπένδ...,2023-01-11,19:33:40,Πολιτικη & Οικονομια,Newsroom,ΤΧΣ: Οι βασικές αρχές της στρατηγικής αποεπένδ...,Πως θα κινηθεί η διαδικασία,Το Ταμείο Χρηματοπιστωτικής Σταθερότητας (ΤΧΣ)...,https://www.athensvoice.gr/images/1074x600/1/j...
637,https://www.athensvoice.gr/epikairotita/diethn...,Με λουλούδια και μάσκες υποδέχονται τους Κινέζ...,2023-01-09,13:23:00,Κοσμος,Newsroom,Στην Ταϊλάνδη οι πρώτοι τουρίστες από την Κίνα...,Χωρίς τεστ covid τελικά η είσοδος στη χώρα,Οι Κινέζοι τουρίστες επέστρεψαν σήμερα Δευτέρα...,https://www.athensvoice.gr/images/1074x600/1/j...
638,https://www.athensvoice.gr/epikairotita/diethn...,Μαζικές συλλήψεις στη Βραζιλία μετά την εισβολ...,2023-01-08,23:24:58,Κοσμος,Newsroom,Εισβολή στο Κογκρέσο της Βραζιλίας - Περισσότε...,Μεγάλες καταστροφές σε κυβερνητικά κτίρια από ...,Στη Βραζιλία τουλάχιστον 150 οπαδοί του ακροδε...,https://www.athensvoice.gr/images/1074x600/1/j...
639,https://www.athensvoice.gr/epikairotita/ellada...,Ανοίγουν αύριο τα σχολεία χωρίς υποχρεωτική μά...,2023-01-08,11:55:00,Ελλαδα,Newsroom,Σχολεία - Κανονικά το άνοιγμα τη Δευτέρα (9/1)...,Πώς σβήνονται οι απουσίες λόγω εποχικής γρίπης...,Τα σχολεία ανοίγουν κανονικά αύριο Δευτέρα (9/...,https://www.athensvoice.gr/images/1074x600/2/j...


In [ ]:
forest_fires_articles_df = fire_articles_df[fire_articles_df['title'].notna() & (fire_articles_df['title'] != '')]
forest_fires_articles_df

,article_url,title,date,time,section,author,excerpt,subtitle,text_body,feat_img_link
0,https://www.athensvoice.gr/epikairotita/diethn...,Μαγικές εικόνες από την «Πόλη του Πάγου» της Κ...,2024-12-29,10:40:50,Κοσμος,Newsroom,Ουρανοξύστες και ναοί φτιαγμένοι από πάγο στο ...,Το πάρκο έχει έκταση ενός εκατομμυρίου τετραγω...,"Διάφορα κτίρια-ορόσημα του Χονγκ Κονγκ, του Μα...",https://www.athensvoice.gr/images/1074x600/jpg...
1,https://www.athensvoice.gr/epikairotita/diethn...,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,2024-12-27,11:26:13,Κοσμος,Newsroom,Καζακστάν: Η Azerbaijan Airlines ανακοίνωσε ότ...,Φόβοι για την ασφάλεια των πτήσεων,Η Azerbaijan Airlines ανακοίνωσε ότι διακόπτει...,https://www.athensvoice.gr/images/1074x600/1/j...
2,https://www.athensvoice.gr/epikairotita/diethn...,Η Ρωσία ξεκίνησε τη χρήση bitcoin για να αντιμ...,2024-12-25,15:02:24,Κοσμος,Newsroom,Ο Ρώσος Πρόεδρος Βλαντιμίρ Πούτιν δήλωσε ότι κ...,Ο Βλαντιμίρ Πούτιν υποστηρίζει την εκτεταμένη ...,Εταιρείες στη Ρωσία ξεκίνησαν να χρησιμοποιούν...,https://www.athensvoice.gr/images/1074x600/1/j...
5,https://www.athensvoice.gr/epikairotita/politi...,Μιλένα Αποστολάκη προς Τάκη Θεοδωρικάκο: Μαζικ...,2024-12-19,10:11:59,Πολιτικη & Οικονομια,Newsroom,Μιλένα Αποστολάκη προς Τάκη Θεοδωρικάκο: Μαζικ...,Η βουλευτής Βόρειου Τομέα του ΠΑΣΟΚ σημειώνει ...,"Το μείζον θέμα των υπέρογκων, διαδοχικών και α...",https://www.athensvoice.gr/images/1074x600/1/j...
6,https://www.athensvoice.gr/life/perivallon/882...,Πόσο αυξήθηκαν οι πυρκαγιές το 2024 - Η ανασκό...,2024-12-17,18:12:54,Περιβαλλον,Newsroom,Πόσο αυξήθηκαν οι πυρκαγιές το 2024 - Η ανασκό...,Εθνικό Αστεροσκοπείο Αθηνών και WWF Ελλάς εγκα...,Η ολοκληρωμένη ανασκόπηση της αντιπυρικής περι...,https://www.athensvoice.gr/images/1074x600/jpg...
...,...,...,...,...,...,...,...,...,...,...
636,https://www.athensvoice.gr/epikairotita/politi...,ΤΧΣ: Οι βασικές αρχές της στρατηγικής αποεπένδ...,2023-01-11,19:33:40,Πολιτικη & Οικονομια,Newsroom,ΤΧΣ: Οι βασικές αρχές της στρατηγικής αποεπένδ...,Πως θα κινηθεί η διαδικασία,Το Ταμείο Χρηματοπιστωτικής Σταθερότητας (ΤΧΣ)...,https://www.athensvoice.gr/images/1074x600/1/j...
637,https://www.athensvoice.gr/epikairotita/diethn...,Με λουλούδια και μάσκες υποδέχονται τους Κινέζ...,2023-01-09,13:23:00,Κοσμος,Newsroom,Στην Ταϊλάνδη οι πρώτοι τουρίστες από την Κίνα...,Χωρίς τεστ covid τελικά η είσοδος στη χώρα,Οι Κινέζοι τουρίστες επέστρεψαν σήμερα Δευτέρα...,https://www.athensvoice.gr/images/1074x600/1/j...
638,https://www.athensvoice.gr/epikairotita/diethn...,Μαζικές συλλήψεις στη Βραζιλία μετά την εισβολ...,2023-01-08,23:24:58,Κοσμος,Newsroom,Εισβολή στο Κογκρέσο της Βραζιλίας - Περισσότε...,Μεγάλες καταστροφές σε κυβερνητικά κτίρια από ...,Στη Βραζιλία τουλάχιστον 150 οπαδοί του ακροδε...,https://www.athensvoice.gr/images/1074x600/1/j...
639,https://www.athensvoice.gr/epikairotita/ellada...,Ανοίγουν αύριο τα σχολεία χωρίς υποχρεωτική μά...,2023-01-08,11:55:00,Ελλαδα,Newsroom,Σχολεία - Κανονικά το άνοιγμα τη Δευτέρα (9/1)...,Πώς σβήνονται οι απουσίες λόγω εποχικής γρίπης...,Τα σχολεία ανοίγουν κανονικά αύριο Δευτέρα (9/...,https://www.athensvoice.gr/images/1074x600/2/j...


In [ ]:
forest_fires_articles_df.to_csv("/content/drive/MyDrive/Final Essay ΕΔΔΕ.2 Αθηνιώτη Μιχαέλα/clean csv/forest_fires_athensvoice_articles_csv.csv", index=False)

In [ ]:
forest_fires_articles_df['text_body'][0]  #έλεγχος ότι πήρε όλο το κείμενο

'Διάφορα κτίρια-ορόσημα του Χονγκ Κονγκ, του Μακάο και της Ταϊβάν, συμπεριλαμβανομένων των ερειπίων του Αγίου Παύλου και του ουρανοξύστη Taipei 101, έκαναν το ντεμπούτο τους στο Harbin Ice-Snow World, ένα από τα πιο εμβληματικά χειμερινά αξιοθέατα της Κίνας\xa0στην επαρχία Heilongjiang. Με θέμα «Dream of Winter, Love among Asia», η φετινή έκδοση του Harbin Ice-Snow World χτίστηκε χρησιμοποιώντας 300.000 κυβικά μέτρα πάγου και χιονιού, ενσωματώνοντας στοιχεία εμπνευσμένα από τους επερχόμενους Ασιατικούς Χειμερινούς Αγώνες του 2025 που θα πραγματοποιηθούν στο Χαρμπίν της Κινας. Με συνολική έκταση ενός εκατομμυρίου τετραγωνικών μέτρων, το πάρκο είναι το μεγαλύτερο στην 26χρονη ιστορία του. Διαθέτει εννέα μεγάλες ζώνες, οι οποίες αναπαριστούν τοπία-ορόσημα 42 χωρών και 3 περιφερειών που είναι μέλη του Ολυμπιακού Συμβουλίου της Ασίας, κατασκευασμένα από πάγο και χιόνι, και εντυπωσιακό νυχτερινό φωτισμό. Δείτε περισσότερες εικόνες: Πηγή φωτογραφιών:\xa0WU HAO/ EPA'

In [ ]:
articles_df['feat_img_link'][0]

'https://www.athensvoice.gr/images/1074x600/jpg/files/2024-12-29/harbin_ice-snow_world__4_.jpg'

In [ ]:
articles_df['article_url'][4]

'https://www.athensvoice.gr/life/geusi/themata/882989/titania-magikes-giortes-me-thea-ti-stolismeni-athina/'